In [74]:
! pip install numpy
! pip install scipy
# you can install packages with the exclamation point

In [75]:
# import all dependencies
import pandas as pd
import numpy as np
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cross_validation import cross_val_score
from scipy.sparse import hstack

In [76]:
# put your functions in one cell, call when you need
def read_tsv(path):
    return pd.read_csv(path, sep='\t')
def get_json_value(col,key):
    return json.loads(col).get(key, '')
def has_text(text,text_val):
    try:
        if 'recipe' in str(text).lower():
            return 1
        else:
            return 0
    except: 
        return 0
    
def check_criteria():
    if(2 >= 2):
        x = 0
    if(2 >= 2):
        x = 1
    else:
        x = 3
    return x # x will equal 1


# def has_text(text,text_val):
#     if( text == None ):      #np.nan is same as None in python
#         return 0
#     if 'recipe' in str(text).lower():
#         return 1
#     else:
#         return 0
#     except: 
#         return 0

!dir


 Volume in drive C is WINDOWS
 Volume Serial Number is AA1C-00BD

 Directory of C:\Users\Zane_Harris\Documents\GitHub\ga-students-datascience\lessons\lesson-13\code\solution-code

04/12/2016  07:46 PM    <DIR>          .
04/12/2016  07:46 PM    <DIR>          ..
04/12/2016  06:36 PM    <DIR>          .ipynb_checkpoints
04/05/2016  06:57 PM            24,129 solution-code-13.ipynb
04/12/2016  07:46 PM            27,676 text_classification_helper-starter.ipynb
               2 File(s)         51,805 bytes
               3 Dir(s)  134,733,430,784 bytes free


In [77]:
path = "../../assets/dataset/stumbleupon.tsv"
data = read_tsv(path)



In [78]:
data.dtypes

url                                object
urlid                               int64
boilerplate                        object
alchemy_category                   object
alchemy_category_score             object
avglinksize                       float64
commonlinkratio_1                 float64
commonlinkratio_2                 float64
commonlinkratio_3                 float64
commonlinkratio_4                 float64
compression_ratio                 float64
embed_ratio                       float64
framebased                          int64
frameTagRatio                     float64
hasDomainLink                       int64
html_ratio                        float64
image_ratio                       float64
is_news                            object
lengthyLinkDomain                   int64
linkwordscore                       int64
news_front_page                    object
non_markup_alphanum_characters      int64
numberOfLinks                       int64
numwords_in_url                   

In [165]:
data.label.head(5)
data.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body,title_length,has_recipe
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,5424,170,8,0.152941,0.079130,0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...,102,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,4973,187,9,0.181818,0.125448,1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...,244,0
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...,79,0
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,2737,120,5,0.041667,0.100858,1,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...,35,0
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,12032,162,10,0.098765,0.082569,0,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...,79,0


In [96]:
# get the title text from the JsonObject 
# note the JsonObject is stored in column boilerplate
# hint: look at function get_json_value above, it takes two inputs, 
# a dataframe column, and a key value to search the Json Dictionary,
# call the new dataframe column "title" 
# hint: first input for get_json_value is a column, 
# and second input 'title'

data['title'] = data.boilerplate.apply(
    lambda x: get_json_value(x, 'title')).fillna('')   #.fillna() because len() below wont run otherwise
data.head()
data.title.head()

0    IBM Sees Holographic Calls Air Breathing Batte...
1    The Fully Electronic Futuristic Starting Gun T...
2    Fruits that Fight the Flu fruits that fight th...
3                  10 Foolproof Tips for Better Sleep 
4    The 50 Coolest Jerseys You Didn t Know Existed...
Name: title, dtype: object

In [97]:
# data['body'] = data.boilerplate.apply(
#     lambda x: get_json_value(x, 'body'))
# data.head()
# data.body.head()


data['title_length'] = data.title.apply(lambda x: len(x) )

data.title_length.head()

0    102
1    244
2     79
3     35
4     79
Name: title_length, dtype: int64

In [122]:
def get_even_numbers(list1):
    cardinality = len(list1)
    even_nums = filter(lambda x: x % 2 == 0, list1)      #if x is divisible by 2 with no remainder then keep, else drop 
    probability = (float(len(even_nums))/cardinality)*100
    return probability

my_list = [1,2,3]

print get_even_numbers(my_list)

33.3333333333


In [123]:
def map_numbers(num):
    if( num % 2 == 0):
        return 1
    return 0

def get_even_numbers(list1):
    cardinality = len(list1)
    even_nums = sum(map(lambda x: map_numbers(x), list1))      #if x is divisible by 2 with no remainder then keep, else drop 
    probability = (float(even_nums)/cardinality)*100
    return probability

my_list = [1,2,3]

print get_even_numbers(my_list)

33.3333333333


In [124]:
# get the body text from the JsonObject
# note the JsonObject is stored in column boilerplate
# hint: look at function get_json_value above, it takes two inputs, 
# a dataframe column, and a key value to search the Json Dictionary,
# call the new dataframe column "body" 
# hint: first input for get_json_value is a column, 
# and second input 'body'

data['body'] = data.boilerplate.apply(
    lambda x: get_json_value(x, 'body')).fillna('')   
data.head()
data.body.head()

0    A sign stands outside the International Busine...
1    And that can be carried on a plane without the...
2    Apples The most popular source of antioxidants...
3    There was a period in my life when I had a lot...
4    Jersey sales is a curious business Whether you...
Name: body, dtype: object

In [145]:
# Now that you have two new columns in your dataframe, one which contains
# only the title text of the website, and the other only body text
# lets see if having the word 'recipe' in the title contributed to higher
# evergreen websites?

# hint, create new column called 'has_recipe', use function has_text,
# which has two inputs, a column, 
# and the specific word you are looking for
# no more hints!

data['has_recipe'] = data.title.apply(lambda text: has_text(text, 'recipe'))
print data.has_recipe.head()

0    0
1    0
2    0
3    0
4    0
Name: has_recipe, dtype: int64


In [146]:
# how many records had the word recipe in the title?
print data[ data.has_recipe == 1 ].shape

(926, 31)


In [147]:
# how many records did not have the word recipe in the title?
print data[ data.has_recipe == 0 ].shape

(6469, 31)


In [148]:
# Interpret Results Above

In [149]:
# remove all null values for title column, by replacing NaN with empty ''
# maintain the 'title' column during the transformation

def clean_value(row):
    if( row == None ):
        return ''
    return row

data['title'] = data.title.fillna('')
data['title'] = data.title.apply(clean_value)

In [150]:
# lets make every word in the title a feature, 
# maybe other words besides recipes contributed to 
# higher evergreen websites?

vectorizer = CountVectorizer(
                max_features = 1000, # max number of words to consider as features
                ngram_range=(1, 2), 
                stop_words='english', # stop when you see word english
                binary=True # each word will be a binary dummy categorical var
            )

In [151]:
# Use `fit` to learn the vocabulary of the titles
vectorizer.fit(data.title)

CountVectorizer(analyzer=u'word', binary=True, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [152]:
# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
x_text_features = vectorizer.transform(data.title)

In [153]:
labels = data.label

In [154]:
model = RandomForestClassifier(n_estimators = 20)
# n_estimators = number of decision trees
# other parameters such as max_depth, best_split, etc available

In [155]:
scores = cross_val_score(
            model,  # your model
            x_text_features,  # your features in vector form
            labels,  # your labels (response/output/predictor)
            scoring='roc_auc' # what metric do you want returned?
        )

In [156]:
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.78536302  0.8069865   0.80275823], Average AUC 0.798369247592


In [157]:
# Identify the features you want from the original dataset
other_features_columns = ['html_ratio', 'image_ratio']
other_features = data[other_features_columns]

In [158]:
# Stack them horizontally together
# This takes all of the word/n-gram columns and appends on two more columns for `html_ratio` and `image_ratio`
features = hstack((x_text_features, other_features)).toarray()

scores = cross_val_score(model, features, labels, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.78346498  0.79800856  0.78835657], Average AUC 0.78994337134


In [168]:
# What features of these are most important, lets actually fit the model!
model.fit(features, labels)
RandomForestClassifier?

In [169]:
all_feature_names = vectorizer.get_feature_names() + other_features_columns

In [170]:
feature_importances = pd.DataFrame(
    {'Features' : all_feature_names, 
     'Importance Score': model.feature_importances_
    }
)

# this makes a nice table for all features, 
# and there corresponding importance

In [171]:
feature_importances.sort_values(
   'Importance Score', 
    ascending=False
).head(10) # sort_values defined for dataframe, default is ascending

,Features,Importance Score
1000,html_ratio,0.156057
1001,image_ratio,0.099260
712,recipe,0.035006
721,recipes,0.017778
190,chocolate,0.013188
180,chicken,0.011046
342,fashion,0.010343
150,cake,0.009739
496,kitchen,0.008796
358,food,0.007795


In [172]:
# Your model was built from features coming from the vocabulary
# of the title vs. the label. 
# Repeat the above process but with your features
# coming from the 'body' column.
# You must fit vocab in your body column, along with creating a vector with
# all possible features in n columns (dummy variables)
# hint: from vectorizor.transform(column_with_text)